In [ ]:
import os
import glob
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import shutil

base_model_path = '/home/gbarbadillo/data/llama-3.1-transformers-8b-instruct-v1'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240724_first_trainings/22_llama31_lr1e-4_1e5dataset_r32/checkpoint-6553'
output_path = '/home/gbarbadillo/data/llama-3.1-count'

base_model_path = "/home/gbarbadillo/data/Phi-3-mini-128k-instruct"
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240724_first_trainings/15_continue_training_phi3_4e5/checkpoint-22800'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240724_first_trainings/22_random_question_lr1e-4_1e5dataset/checkpoint-6228'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240729_arc_fine_tuning/01_phi-3_3epochs_train_lr2e-4/checkpoint-400'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240729_arc_fine_tuning/04_phi-3_3epochs_train_lr2e-4_even_more_augmentation/checkpoint-1090'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240729_arc_fine_tuning/06_phi-3_add_color_swaps_continue_2/checkpoint-2898'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240729_arc_fine_tuning/02_phi-3_24epochs_train_lr2e-4_no_augmentation_continue_3/checkpoint-288'
# lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240729_arc_fine_tuning/11_phi-3_overfit-to-train_continue_lr1e-5_longer/checkpoint-1200'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240729_arc_fine_tuning/10_phi-3_1rearc100_2train_lr5e-5_color-swap-no-preserve_continue/checkpoint-1000/'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240802_test_time_fine-tuning/01_baseline_no_data_augmentation_lr5e-5/checkpoint-150'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240802_test_time_fine-tuning/03_data_augmentation_color_swap_lr5e-5_5e3steps/checkpoint-5000'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240802_test_time_fine-tuning/02_data_augmentation_lr5e-5_3e3steps/checkpoint-2400'
output_path = "/home/gbarbadillo/data/Phi-3-arc"

base_model_path = '/home/gbarbadillo/data/Qwen2-0.5B-Instruct'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240809_smaller_models/04_Qwen2-0.5B-Instruct_lr2e-4_r128_8e3steps/checkpoint-6700'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240809_smaller_models/04_Qwen2-0.5B-Instruct_lr5e-5_r128_8e3steps/checkpoint-4900'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240812_smaller_models_test-time_fine-tuning/01_Qwen2-0.5B-Instruct_lr1e-4_r128_2e3steps/checkpoint-2000'
output_path = "/home/gbarbadillo/data/Qwen2-0.5B-arc"

base_model_path = '/home/gbarbadillo/data/Qwen2-1.5B-Instruct'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240809_smaller_models/05_Qwen2-1.5B-Instruct_lr1e-4_r32_8e3steps/checkpoint-4000'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240812_smaller_models_test-time_fine-tuning/03_Qwen2-1.5B-Instruct_lr1e-4_r32_2e3steps_c/checkpoint-2000'
output_path = "/home/gbarbadillo/data/Qwen2-1.5B-arc"

base_model = AutoModelForCausalLM.from_pretrained(base_model_path, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
if 'llama' in base_model_path:
    tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
    base_model.resize_token_embeddings(len(tokenizer))


model = PeftModel.from_pretrained(base_model, lora_path)
merged_model = model.merge_and_unload()
print('Saving the merged model to the output path')
merged_model.save_pretrained(output_path)

for filepath in glob.glob(os.path.join(base_model_path, '*')):
    dst = os.path.join(output_path, os.path.basename(filepath))
    if not os.path.exists(dst):
        print('Copying', filepath)
        shutil.copy(filepath, dst)

print('Done!')